##### Import requirements

In [130]:
import os
import json
import pandas as pd

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

##### CREATE YOUTUBE API FROM GOOGLE APIs

In [77]:
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

#login google
def login():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    #create OAuth 2.0 archive at https://console.cloud.google.com/apis/credentials and download the files
    client_secrets_file = "secret_file.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_local_server()
    #credentials = flow.run_console()
    
    return credentials

#create youtube api
credentials = login()
api_service_name = "youtube"
api_version = "v3"
youtube = googleapiclient.discovery.build(api_service_name, api_version, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=743859723925-rf5c7ehraeo0hpkjqhopf4ditnqgm7fa.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=twsARFufDlEX8jGlDqK2WowFHrIHZk&access_type=offline


##### REQUEST DATA TO THE API

In [79]:
def youtubeSearch(nextPageToken, max):

    #search to get the IDs of the videos
    request = youtube.search().list(
        part="snippet",
        maxResults=max,
        pageToken=nextPageToken,
        publishedAfter="2022-10-02T21:00:00Z",
        publishedBefore="2022-10-29T21:59:59Z",
        q="eleição",
        regionCode="BR",
        type="video"
    )

    response = request.execute()

    #Concat the IDs into a unique string separated by comma
    listOfIDs = response['items'][0]['id']['videoId']
    for i in range(1, len(response['items'])):
        listOfIDs = listOfIDs + "," + response['items'][i]['id']['videoId']

    #Request the data from the vidos os the list of IDs
    request = youtube.videos().list(
        part="snippet,contentDetails,statistics", 
        id=listOfIDs
    )
    
    data = request.execute()

    return response, data

Each request return a limited number of items, then it is necessary save the nextPageToken.

With this nextPageToken, it is possible to make a new resquest, but then starting ahead in the resulted array.

After all requests, it is possible to concat all the results.

In [ ]:
maxResults = 100
nextPageToken = ""
stopFlag = False

count = 0
listOfResponse = []
listOfData = []
while(count<maxResults):

    if stopFlag:
        break

    response, data = youtubeSearch(nextPageToken, maxResults)

    for key in list(response.keys()):
        if key == 'nextPageToken':
            nextPageToken = str(response['nextPageToken'])
            stopFlag = False
            break
        else:
            stopFlag = True

    count = count + len(response['items'])

    listOfData = listOfData + [data]

data = listOfData[0]['items'] 
for i in range(1,len(listOfData)):
    data = data + listOfData[i]['items']

Salva resultado em json

In [117]:
#If concat gets right
if len(data) == maxResults:
    with open('./result/ex1/videos.json', 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file,ensure_ascii=False)

Abre resultado

In [131]:
#f = open('./result/ex1/videos.json')
#data = json.load(f)
#f.close()

Salva resultado em csv

In [168]:
df = pd.read_json("videos.json")

dfSnippet = pd.json_normalize(df['snippet'])
df = pd.concat([df,dfSnippet],axis=1).drop(columns=['snippet'])

dfcontentDetails = pd.json_normalize(df['contentDetails'])
df = pd.concat([df,dfcontentDetails],axis=1).drop(columns=['contentDetails'])

dfStatistics = pd.json_normalize(df['statistics'])
df = pd.concat([df,dfStatistics],axis=1).drop(columns=['statistics'])

df.to_csv('./result/ex1/videos.csv', sep=";")

##### Informações do video mais comentado no periodo do segundo turno de 2022, a partir da busca por "eleição" no youtube

In [169]:
df['commentCount'] = df['commentCount'].fillna("0")
df['commentCount'] = df['commentCount'].astype(int)
df = df.sort_values(by='commentCount', ascending=False)

#resultado
print(df.iloc[0])

kind                                                              youtube#video
etag                                                FCBcMzqHy6XMPClE4kUSdTP1iTI
id                                                                  5BprpVbXCc0
publishedAt                                                2022-10-22T21:00:01Z
channelId                                              UCGPCO1PVeWZ-egT6F2wKnaw
title                         ELEIÇÕES DE VÓ - PARTE 2 - OS DEPOIMENTOS #ele...
description                   ELEIÇÕES DE VÓ - PARTE 2 - OS DEPOIMENTOS #ele...
channelTitle                                                 O Victor Magalhães
tags                          [carlos matrix, carlos trafico de brigadeiro, ...
categoryId                                                                   24
liveBroadcastContent                                                       none
defaultAudioLanguage                                                         pt
thumbnails.default.url           https:/

In [171]:
#pip freeze > requirements.txt